In [1]:
! pip install transformers==4.6.0
! pip install emoji==0.6.0
!pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
! pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install sentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re

In [79]:
data = pd.read_csv("/content/drive/MyDrive/data/dataset.csv")
text = data[['tweet_text', 'tweet_date']]
pd.options.mode.chained_assignment = None

In [80]:
# remove all links
text['tweet_text'] = text['tweet_text'].str.replace(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', regex=True)

# remove all line breaks 
text['tweet_text'] = text['tweet_text'].str.replace(r'\n', '', regex=True)    
# text['tweet_text'] = text['tweet_text'].str.replace(r'\t', '', regex=True)
text['tweet_text'] = text['tweet_text'].str.replace(r'\r', '', regex=True)

# remove all emojis
text['tweet_text'] = text['tweet_text'].str.replace(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", '', flags=re.UNICODE,regex=True)


# Remove double fullstops
text['tweet_text'] = text['tweet_text'].str.replace(r'\.+', '.', regex=True)



text['tweet_text'] = text['tweet_text'].str.replace(r"\.(?=\S)", '.', regex=True)

#Remove &amp sign
text['tweet_text'] = text['tweet_text'].str.replace(r"&amp;", 'and', regex=True)

text['tweet_text'] = text['tweet_text'].str.replace(r'\â€”', 'and', regex=True)



In [81]:
text = text[text['tweet_text'].notna()]
text['tweet_date'] = pd.to_datetime(text['tweet_date'], utc=True)
text['Date'] = text['tweet_date'].dt.date
text = text.rename(columns={'tweet_text': 'Text'})
text = text.drop('tweet_date', axis=1)

In [83]:
def find_at_word(text):
    
    word=re.findall(r'(?<=@)\w+',text)
    return " @".join(word)



def add(text):
    if len(text) > 1:
        return "@" + text
    else:
        return text

    
text['Usernames']=text['Text'].apply(lambda x: find_at_word(x))
text['Usernames']=text['Usernames'].apply(lambda x: add(x))


In [84]:
text['Text'] = text['Text'].str.replace(r'/\S*@\S*\s?/g', '', regex=True)  # remove all emails

# Remove all usernames from the tweets column for better interpretability, AFTER ADDING TO ANOTHER COLUMN
# text['Text'] = text['Text'].str.replace(r'([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)', '', regex=True)  # remove all emails


# Instead of the upper two regexes, lets use this. Itll keep the user names but just remove the @ symbol
text['Text'] = text['Text'].str.replace('@', '', regex=True)
text['Text']=text['Text'].str.capitalize()

In [85]:

text.head(5)


,Text,Date,Usernames
1,Arikring therealnumber6 risepdx jackthelad1947...,2022-09-30,@arikring @TheRealNumber6 @RisePDX @Jackthelad...
2,Californians will not be fooled by gov. newsom...,2022-09-30,
3,#usa renewables production hit record high dri...,2022-09-30,@climateguyw @arikring
4,Euro area inflation hit a record 10% in septem...,2022-09-30,
5,Blairmcdougall here is a sample of your lies i...,2022-09-30,@blairmcdougall


# NER MODEL [FROM HERE](https://huggingface.co/Jean-Baptiste/camembert-ner)

In [2]:
! pip install transformers

In [66]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("TweebankNLP/bertweet-tb2_wnut17-ner")

model = AutoModelForTokenClassification.from_pretrained("TweebankNLP/bertweet-tb2_wnut17-ner")

nlp = pipeline('ner', model=model, tokenizer = tokenizer)


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [76]:
text['Text'].dropna(inplace=True)
text['Text'].drop_duplicates(inplace=True)

In [88]:
# Create a function that will take in a row of tweet text from the dataframe, predict the entities,
# and store the result to another column of the dataframe.

def predict_NER(row):
  return nlp(row)

text['Entities']=text['Text'].apply(lambda i: predict_NER(i))

In [ ]:
text.to_csv("latest_download.csv", index=False)

# Save the file after generating the entities, since it took soo long, and then load it in a new dataframe

In [ ]:
new_Df = pd.read_csv("/content/latest_download.csv", index=False)

In [16]:
new_Df

,Unnamed: 0,Text,Date,Usernames,Entities
0,1,Arikring therealnumber6 risepdx jackthelad1947...,2022-09-30,@arikring @TheRealNumber6 @RisePDX @Jackthelad...,"[{'entity': 'B-PER', 'score': 0.9106537, 'inde..."
1,2,Californians will not be fooled by gov. newsom...,2022-09-30,NaN,"[{'entity': 'B-LOC', 'score': 0.9050666, 'inde..."
2,3,#usa renewables production hit record high dri...,2022-09-30,@climateguyw @arikring,"[{'entity': 'B-PER', 'score': 0.6425788, 'inde..."
3,4,Euro area inflation hit a record 10% in septem...,2022-09-30,NaN,"[{'entity': 'B-MISC', 'score': 0.8712144, 'ind..."
4,5,Blairmcdougall here is a sample of your lies i...,2022-09-30,@blairmcdougall,"[{'entity': 'B-PER', 'score': 0.95930564, 'ind..."
...,...,...,...,...,...
10030,10031,Boeufblogginon steveirons patskarvelas he prob...,2022-08-13,@Boeufblogginon @steveirons @PatsKarvelas,"[{'entity': 'B-PER', 'score': 0.96054596, 'ind..."
10031,10032,As we find new ways to help people prepare for...,2022-08-13,NaN,"[{'entity': 'B-PER', 'score': 0.9605521, 'inde..."
10032,10033,This is what the cost of living crisis looks l...,2022-08-13,NaN,"[{'entity': 'B-LOC', 'score': 0.7147742, 'inde..."
10033,10034,#nuclearenergy can point the way out of the en...,2022-08-13,NaN,[]


In [6]:
# In this cell, we are appending the NER predictions of the model for each row of the dataframe into a list as a dictionary

NER_results=[]
for i in new_Df['Entities']:  
  NER_results.append(eval(i))

In [8]:
# Here we are creating a dictionary that will have a list for each item in NER_results. Inside the list, we will select the word parameter of the NER predictions

entities_dictionary = dict()
i=0
for row in NER_results:
  entities_dictionary[i] = []
  count = 0

  for entity in row:
    entities_dictionary[i].append({count:entity['word']})
    count+=1
  i+=1


In [ ]:
# Filtration by date
def filtered_by_date(date):
  x = new_Df[(new_Df['Date'] == date)][['Text']]
  y = x.values
  return '.'.join(str(item) for innerlist in y for item in innerlist)


filtered_texts = filtered_by_date('2022-09-24')
filtered_texts

# SUMMARIZATON MODEL [FROM HERE](https://huggingface.co/facebook/bart-large-cnn)

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [12]:
# Create a function that will take in each row of tweet from the dataframe nd generate a summary of that tweet.
# Store the summary in another column

def summarize_each_tweet(row):
  token = tokenizer(row, truncation=True, padding="longest", return_tensors="pt")
  summary = model.generate(**token)
  res = tokenizer.decode(summary[0])
  return res


# text['Summary']=text['Text'].apply(lambda i: summarize_each_tweet(i))
summary = summarize_each_tweet(filtered_texts)

In [13]:

summary

'<pad> Labour Party leader Ed Miliband has called for a "new era of economic growth and permanently lower energy bills by turning the UK into an independent green "superpower" before 2030, through a massive expansion of wind and solar energy.</s>'